In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 
# Processing of an ega list: Restructure the csv table for further processing.
# 
# 
# 
#
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# ~~~~~ Importing the standard modules  ~~~~~

# import sys

# import numpy as np
# import pandas as pd                                  # ~~~~~  important !!!  ~~~~~~
# from pandas import Series, DataFrame

# # for editing Excel files is required: 
# import xlrd, openpyxl

# # A data frame can be added as a new sheet to an existing excel sheet. 
# # For this operation, the library required is openpyxl.
# from openpyxl import load_workbook

# # import for JSON - JavaScript Object Notation
# import json

# # import requests for web access
# import requests

# # BeautifulSoup also for web access

# import bs4
# from bs4 import BeautifulSoup

# import datetime
# from datetime import date

# #https://de.wikipedia.org/wiki/Algorithmus
# # website = requests.get('https://de.wikipedia.org/wiki/')
# # soup = BeautifulSoup(website.content, 'html.parser')

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#!pip install -r requirements.txt

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Processing of an ega list: Restructure the csv table for further processing.
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Author:  Dr. Klaus-Peter Fichtner 
## Date: 2022-12-29

In [ ]:
import pandas as pd

# Lese die CSV-Datei "Test1 VB10 EGA v03.csv" in ein Pandas-DataFrame ein
ega = pd.read_csv("Test1 VB10 EGA v03.csv", sep=";", encoding="latin1")

FileNotFoundError: ignored

In [ ]:
ega.head()

In [ ]:
ega.info()

In [ ]:
# Lösche alle Spalten hinter Spalte 12
ega = ega.drop(ega.columns[13:], axis=1)
ega.describe()

In [ ]:
ega.info()

In [ ]:
ega.head()

In [ ]:
# Ersetze alle NaN-Werte in der Spalte 'Hazard Type' durch den Wert 'Chapter'
ega['Hazard Type'] = ega['Hazard Type'].fillna('Chapter')

In [ ]:
# Finde die Measures in der der ega-Tabelle
measure_row = ega.loc[ega['Hazard Type'] == 'Measure', :]
index_measure = measure_row.index[0]


# Finde den Index der ersten Zeile mit Hazard Type = 'Chapter' vor einer Zeile mit Hazard Type = 'Measure'
index_chapter = None
for index, row in ega.iterrows():
    if row['Hazard Type'] == 'Measure':
        break
    if row['Hazard Type'] == 'Chapter':
        index_chapter = index

# Finde den Index der ersten Zeile mit Hazard Type = 'Function' vor einer Zeile mit Hazard Type = 'Measure'
index_function = None
for index, row in ega.iterrows():
    if row['Hazard Type'] == 'Measure':
        break
    if row['Hazard Type'] == 'Function':
        index_function = index

# Finde den Index der ersten Zeile mit Hazard Type = 'Hazard' vor einer Zeile mit Hazard Type = 'Measure'
index_hazard = None
for index, row in ega.iterrows():
    if row['Hazard Type'] == 'Measure':
        break
    if row['Hazard Type'] == 'Hazard':
        index_hazard = index

# Finde den Index der ersten Zeile mit Hazard Type = 'Cause' vor einer Zeile mit Hazard Type = 'Measure'
index_cause = None
for index, row in ega.iterrows():
    if row['Hazard Type'] == 'Measure':
        break
    if row['Hazard Type'] == 'Cause':
        index_cause = index


print('index(chapter) = ' + str(index_chapter))
print('index(function) = ' + str(index_function))
print('index(hazard) = ' + str(index_hazard))
print('index(cause) = ' + str(index_cause))
print('index(measure) = ' + str(index_measure))

In [ ]:
# Index-Paare erstellen - dazu zuerst eine leere Liste erstellen:
index_pairs = []
for i, row in ega.iterrows():
    if row["Hazard Type"] == "Measure":
        index_measure = i
        for j in range(index_measure, 0, -1):
            if ega.iloc[j]["Hazard Type"] == "Chapter":
                index_chapter = j
                break
        for k in range(index_measure, index_chapter, -1):
            if ega.iloc[k]["Hazard Type"] == "Function":
                index_function = k
                break
        for l in range(index_measure, index_function, -1):
            if ega.iloc[l]["Hazard Type"] == "Hazard":
                index_hazard = l
                break
        for m in range(index_measure, index_hazard, -1):
            if ega.iloc[m]["Hazard Type"] == "Cause":
                index_cause = m
                break
        index_pairs.append((index_chapter, index_function, index_hazard, index_cause, index_measure))
        # break  # entferne diese Zeile, wenn du alle Measure-Zeilen finden möchtest
# print(index_pairs)

In [ ]:
 print(index_pairs)

In [ ]:
result_table = pd.DataFrame(columns=['Chapter ID', 'Function ID', 'Hazard ID', 'Cause ID', 'Measure ID'])

for index_tuple in index_pairs:
  id_chapter = ega['ID'][index_tuple[0]]
  id_function = ega['ID'][index_tuple[1]]
  id_hazard = ega['ID'][index_tuple[2]]
  id_cause = ega['ID'][index_tuple[3]]
  id_measure = ega['ID'][index_tuple[4]]
  result_table = result_table.append({'Chapter ID': id_chapter, 'Function ID': id_function, 'Hazard ID': id_hazard, 'Cause ID': id_cause, 'Measure ID': id_measure}, ignore_index=True)

    
# Wandle alle Spaltenwerte in Integers um
result_table = result_table.astype({'Chapter ID': 'int32', 'Function ID': 'int32', 'Hazard ID': 'int32', 'Cause ID': 'int32', 'Measure ID': 'int32'})
# print(result_table)

In [ ]:
# Erstelle Tabelle mit den IDs, Types, Requirements, Descriptions
result_table = pd.DataFrame(columns=['Chapter ID', 'Chapter Type', 'Chapter Req', 
                                     'Function ID', 'Function Type', 'Function Req',
                                     'Hazard ID','Hazard Type', 'Hazard Req',
                                     'Cause ID', 'Cause Type', 'Cause Req',
                                     'Measure ID', 'Measure Type', 'Measure Req','Measure Description',])

# Iteriere über alle index_pairs und füge die IDs, Types, Requirements, Descriptions der Tabelle hinzu
for index_tuple in index_pairs:
  id_chapter = int(ega['ID'][index_tuple[0]])
  id_function = int(ega['ID'][index_tuple[1]])
  id_hazard = int(ega['ID'][index_tuple[2]])
  id_cause = int(ega['ID'][index_tuple[3]])
  id_measure = int(ega['ID'][index_tuple[4]])

  ht_chapter = ega['Hazard Type'][index_tuple[0]]
  ht_function = ega['Hazard Type'][index_tuple[1]]
  ht_hazard = ega['Hazard Type'][index_tuple[2]]
  ht_cause = ega['Hazard Type'][index_tuple[3]]
  ht_measure = ega['Hazard Type'][index_tuple[4]]

  req_chapter = ega['Requirement'][index_tuple[0]]
  req_function = ega['Requirement'][index_tuple[1]]
  req_hazard = ega['Requirement'][index_tuple[2]]
  req_cause = ega['Requirement'][index_tuple[3]]
  req_measure = ega['Requirement'][index_tuple[4]]
  
  desc_chapter = ega['Description'][index_tuple[0]]
  desc_function = ega['Description'][index_tuple[1]]
  desc_hazard = ega['Description'][index_tuple[2]]
  desc_cause = ega['Description'][index_tuple[3]]
  desc_measure = ega['Description'][index_tuple[4]]

  result_table = result_table.append(
          {
          'Chapter ID': id_chapter, 'Chapter Type': ht_chapter, 'Chapter Req': req_chapter, 
          'Function ID': id_function, 'Function Type': ht_function, 'Function Req': req_function, 
          'Hazard ID': id_hazard, 'Hazard Type': ht_hazard,'Hazard Req': req_hazard, 
          'Cause ID': id_cause,'Cause Type': ht_cause, 'Cause Req': req_cause, 
          'Measure ID': id_measure,'Measure Type': ht_measure, 'Measure Req': req_measure, 'Measure Description': desc_measure, 
          },
          ignore_index=True)

# Wandle alle Spaltenwerte in Integers um
# #result_table = result_table.astype({'Chapter ID': 'int32', 
#                                     'Function ID': 'int32', 
#                                     'Hazard ID': 'int32', 
#                                     'Cause ID': 'int32', 
#                                     'Measure ID': 'int32'})
# print(result_table)

In [ ]:
result_table.to_csv("result_table.csv", sep=";", encoding="utf-8")

In [ ]:
# Lese die CSV-Datei "result_table.csv" in ein Pandas-DataFrame ein
restab = pd.read_csv("result_table.csv", sep=";", encoding="latin1")

In [ ]:
restab.head(10)